In [74]:
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier
import numpy as np

def predicted_bugNum_vs_true_bugNum(Y_test_predict,Y_test_bugs):
    n = 0
    bugrowcount = 0
    for i in range(0,len(Y_test_predict)):
        if(Y_test_bugs[i]!=False):
            bugrowcount += 1
            if(Y_test_predict[i]==Y_test_bugs[i]):
                n=n+1
#    print(n/bugrowcount,n,bugrowcount)
    return n/bugrowcount,n

def load_one_version_method_matrix(matrix_path):
    print(matrix_path)
    return pd.read_csv(matrix_path)

def get_selected_rows(columncombation,X_data):
    flag = False
    for i in columncombation:
        if(flag == False):
            c = X_data[:,i].T
            flag = True
        else:
            c = np.vstack((c,X_data[:,i].T))
    X_train_selected = c.T 
    return X_train_selected

def classification_binary(csvpath,startColumnName,
                          endColumnName,isdefects,importance_result,
                         bugcolumnName,columncombation_j = None):

    filePath = "AlltheMatrix/"

    filePathNowusing = filePath + csvpath

    changed_matrix = load_one_version_method_matrix(filePathNowusing)
    print(f'lens: {len(changed_matrix.columns.values)}')
    matrixlen = int(len(changed_matrix))
    startindex = 0
    endindex = 0
    for i, column in enumerate(changed_matrix.columns.values):
        if(column == startColumnName):
            startindex = i
        elif (column == endColumnName):
            endindex = i
    print(f'start:{startindex},end:{endindex}') 
    target = changed_matrix[bugcolumnName].values
    data = changed_matrix.values[:,startindex+1:endindex]
    print(target.shape)
    print(data.shape)

    #shuffleIndex
    import numpy as np
    shuffle_index = np.random.permutation(matrixlen)
    target, data = target[shuffle_index], data[shuffle_index]

    #shuffleIndex
    import numpy as np
    shuffle_index = np.random.permutation(matrixlen)
    target, data = target[shuffle_index], data[shuffle_index]
    print(type(data))
    print("@@@@@@@@@@@@@@@@@@@@@@@@@@")
    #split the train set and the test set
    X_train = data[:round(0.7*matrixlen)]
    X_test = data[round(0.7*matrixlen):]
    Y_train = target[:round(0.7*matrixlen)]
    Y_test = target[round(0.7*matrixlen):]
    print(X_train,Y_train.shape)
    print("@@@@@@@@@@@@@@@@@@@@@@@@@@")
    #train a binary classifier
    Y_train_bugs = (Y_train > isdefects)
    Y_test_bugs = (Y_test > isdefects)
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import cross_val_score

    if(columncombation_j!=None):
        print("####################################")
        X_train = get_selected_rows(columncombation_j,X_train)
        X_test = get_selected_rows(columncombation_j,X_test)

    forest_clf = RandomForestClassifier(n_estimators = 200)
    forest_clf.fit(X_train,Y_train_bugs)

    cross_result = cross_val_score(forest_clf, X_train, Y_train_bugs, cv = 5, scoring = 'accuracy')
    recall_result = cross_val_score(forest_clf, X_train, Y_train_bugs, cv = 5, scoring = 'recall')
    Y_forest_predict = forest_clf.predict(X_train)

    from sklearn.metrics import precision_score, recall_score
    forst_precision = precision_score(Y_train_bugs, Y_forest_predict)
    forst_recall = recall_score(Y_train_bugs, Y_forest_predict)
    print(f'Forest_accurancy is {forst_precision}')
    print(f'Forest_recall is {forst_recall}')
    print(cross_result)
    #    print(X_train_selected)
    Y_test_predict = forest_clf.predict(X_test)

    findingRate = predicted_bugNum_vs_true_bugNum(Y_test_predict,Y_test_bugs)
    print(f'fingdingrate = {findingRate}')
    
    # get the importance of each column
    importances=forest_clf.feature_importances_
    print('the importance of the each column：\n',importances)
    indices = np.argsort(importances)[::-1]
    print('the descend order for importance of the column：\n',indices)
    most_import = indices[:10]
    print(f'most importtant columns:{most_import}')
    print(X_train[:,most_import])
    

    columnlist = np.arange(0,endindex-startindex-1,1)
    for i in columnlist:
        if i in most_import:
            importance_result[i] +=1
    return cross_result,recall_result

In [102]:
def testdorelcipse(endcolumnname):
    filePath = "AlltheMatrix/"

    filePathNowusing = filePath + "modified_eclipse-metrics-packages-3.0.csv"

    changed_matrix = load_one_version_method_matrix(filePathNowusing)
    print(f'lens: {len(changed_matrix.columns.values)}')
    matrixlen = int(len(changed_matrix))
    startindex = 0
    endindex = 0
    for i, column in enumerate(changed_matrix.columns.values):
        if(column == 'ACD_max'):
            startindex = i
        elif (column == endcolumnname):
            endindex = i
    print(f'start:{startindex},end:{endindex}') 
    target = changed_matrix["post"].values
    data = changed_matrix.values[:,startindex+1:endindex]
    print(target.shape)
    print(data.shape)

    #shuffleIndex
    import numpy as np
    shuffle_index = np.random.permutation(matrixlen)
    target, data = target[shuffle_index], data[shuffle_index]
    print(type(data))
    print("@@@@@@@@@@@@@@@@@@@@@@@@@@")
    #split the train set and the test set
    X_train = data
    Y_train = target
    print(X_train,Y_train.shape)
    print("@@@@@@@@@@@@@@@@@@@@@@@@@@")
    #train a binary classifier
    Y_train_bugs = (Y_train > 0)
#     print(Y_train_bugs)
#     print(X_train)
    
    
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import cross_val_score
    forest_clf = LogisticRegression(solver='liblinear',max_iter=2000,class_weight={0:0.85,1:1})
    forest_clf.fit(X_train,Y_train_bugs)
    cross_result = cross_val_score(forest_clf, X_train, Y_train_bugs, cv = 5, scoring = 'accuracy')
    recall_result = cross_val_score(forest_clf, X_train, Y_train_bugs, cv = 5, scoring = 'recall')
    
#     filePathNowusing = filePath + "modified_eclipse-metrics-files-2.1.csv"

#     changed_matrix = load_one_version_method_matrix(filePathNowusing)
#     print(f'lens: {len(changed_matrix.columns.values)}')
#     matrixlen = int(len(changed_matrix))
#     startindex = 0
#     endindex = 0
#     for i, column in enumerate(changed_matrix.columns.values):
#         if(column == 'ACD'):
#             startindex = i
#         elif (column == endcolumnname):
#             endindex = i
#     print(f'start:{startindex},end:{endindex}') 
#     target = changed_matrix["post"].values
#     data = changed_matrix.values[:,startindex+1:endindex]
#     print(target.shape)
#     print(data.shape)

#     #shuffleIndex
#     import numpy as np
#     shuffle_index = np.random.permutation(matrixlen)
#     target, data = target[shuffle_index], data[shuffle_index]
#     print(type(data))
#     print("@@@@@@@@@@@@@@@@@@@@@@@@@@")
#     #split the train set and the test set
#     X_train = data
#     Y_train = target
#     print(X_train,Y_train.shape)
#     print("@@@@@@@@@@@@@@@@@@@@@@@@@@")
#     #train a binary classifier
#     Y_train_bugs = (Y_train > 0)
    
    
    Y_forest_predict = forest_clf.predict(X_train)
    from sklearn.metrics import precision_score, recall_score
    forst_precision = precision_score(Y_train_bugs, Y_forest_predict)
    forst_recall = recall_score(Y_train_bugs, Y_forest_predict)
    print(f'Forest_accurancy is {forst_precision}')
    print(f'Forest_recall is {forst_recall}')

    return forst_precision,forst_recall

In [103]:
a = testdorelcipse("NORM_InstanceofExpression")

AlltheMatrix/modified_eclipse-metrics-packages-3.0.csv
lens: 212
start:6,end:190
(661,)
(661, 183)
<class 'numpy.ndarray'>
@@@@@@@@@@@@@@@@@@@@@@@@@@
[[0.0 13.333333333333336 49.0 ... 0.0430786904078116 0.019529006318207927
  0.0011487650775416428]
 [0.0 0.8666666666666667 9.0 ... 0.029117379435850774
  0.004549590536851683 0.0]
 [35.0 6.708074534161492 84.0 ... 0.027932086691182224
  0.017213050622017056 0.00031296455676394646]
 ...
 [5.0 3.2962962962962963 42.0 ... 0.021655172413793104
  0.014620689655172412 0.0008275862068965517]
 [16.0 6.307017543859649 34.0 ... 0.02437835202340322 0.01718673817649927
  0.0]
 [2.0 2.4482758620689653 13.0 ... 0.013681592039800995
  0.010416666666666666 0.0]] (661,)
@@@@@@@@@@@@@@@@@@@@@@@@@@
Forest_accurancy is 0.8269896193771626
Forest_recall is 0.7635782747603834


In [81]:
print(f'Logistic_accurancy is {sum(a[0])/5}')
print(f'Logistic_recall is {sum(a[1])/5}')

TypeError: 'numpy.float64' object is not iterable